In [ ]:
from sklearn import datasets
import pandas as pd
import numpy as np

In [ ]:
!wget https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv

--2022-05-24 16:39:14--  https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60302 (59K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  58.89K  --.-KB/s    in 0.007s  

2022-05-24 16:39:14 (8.59 MB/s) - ‘titanic.csv’ saved [60302/60302]



In [42]:
df = pd.read_csv('titanic.csv', header = 0)
df_filtered = df[['Survived','Pclass','Sex','Age','Fare','Embarked']]
df_filtered['Age'].fillna(df_filtered['Age'].mean(), inplace = True)
df_filtered['Embarked'].fillna('S', inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [41]:
df_features = df_filtered
genders = {"male": 0, "female": 1}
df_features["Sex"] = df_filtered["Sex"].apply(lambda s: genders.get(s))
embarkments = {"S": 0, "C": 1, "Q": 2}
df_features["Embarked"] = df_features["Embarked"].fillna("U").apply(lambda e: embarkments.get(e))
df_features = df_features.to_numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
X = df_features[:, 1:6]
y = df_features[:, 0].astype(np.int64)

In [218]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

In [260]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

**Персептрон**

In [301]:
from sklearn.linear_model import Perceptron
ppn = Perceptron(eta0=0.001, random_state=5)
ppn.fit(X_train_std, y_train)

Perceptron(eta0=0.001, random_state=5)

In [309]:
from sklearn.metrics import accuracy_score
y_pred = ppn.predict(X_test_std)
print('Число ошибочно классифицированных образцов: %d' % (y_test != y_pred).sum())
print('Верность: %.3f' % accuracy_score(y_test, y_pred))

Число ошибочно классифицированных образцов: 64
Верность: 0.761


**Логистическая регрессия**

In [303]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=100.0, random_state=1, solver='lbfgs', multi_class='ovr')
lr.fit(X_train_std, y_train)

LogisticRegression(C=100.0, multi_class='ovr', random_state=1)

In [304]:
y_pred = lr.predict(X_test_std)
print('Число ошибочно классифицированных образцов: %d' % (y_test != y_pred).sum())
print('Верность: %.3f' % accuracy_score(y_test, y_pred))

Число ошибочно классифицированных образцов: 52
Верность: 0.806


**Классификация с максимальным зазором на основе метода опорных векторов**

In [254]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf', gamma=1.0, C=1.0, random_state = 1)
svm.fit(X_train_std, y_train)

SVC(gamma=1.0, random_state=1)

In [255]:
y_pred = svm.predict(X_test_std)
print('Число ошибочно классифицированных образцов: %d' % (y_test != y_pred).sum())
print('Верность: %.3f' % accuracy_score(y_test, y_pred))

Число ошибочно классифицированных образцов: 50
Верность: 0.813


**Обучение на основе деревьев решений**

In [251]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=1, random_state = 17)
clf.fit(X_train_std, y_train)

DecisionTreeClassifier(max_depth=1, random_state=17)

In [252]:
y_pred = clf.predict(X_test_std)
print('Число ошибочно классифицированных образцов: %d' % (y_test != y_pred).sum())
print('Верность: %.3f' % accuracy_score(y_test, y_pred))

Число ошибочно классифицированных образцов: 53
Верность: 0.802


**Алгоритм ленивого обучения k-ближайших соседей**

In [249]:
from sklearn.neighbors import KNeighborsClassifier
ngbh = KNeighborsClassifier(n_neighbors=30)
ngbh.fit(X_train_std, y_train)

KNeighborsClassifier(n_neighbors=30)

In [250]:
y_pred = ngbh.predict(X_test_std)
print('Число ошибочно классифицированных образцов: %d' % (y_test != y_pred).sum())
print('Верность: %.3f' % accuracy_score(y_test, y_pred))

Число ошибочно классифицированных образцов: 49
Верность: 0.817
